<a href="https://colab.research.google.com/github/joaoAngelo2/similaridade/blob/main/classificacao_similaridade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
from time import time

import numpy as np
from IPython.display import Markdown, display
from matplotlib import pyplot as plt

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import tensorflow as tf

try:
    import tensorflow_similarity as tfsim
except ModuleNotFoundError:
    !pip install tensorflow_similarity
    import tensorflow_similarity as tfsim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.6.1-py2.py3-none-any.whl.metadata (8.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.4/230.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 9.3 MB/s eta 0:00:00
Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.5 MB/s eta 0:00:00
ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'tensorflow_similarity'

In [3]:
IMG_SIZE = 300

def resize(img, label):
    with tf.device("/cpu:0"):
        img = tf.cast(img, dtype="int32")
        img = tf.image.resize_with_pad(img, IMG_SIZE, IMG_SIZE)
        return img, label

In [4]:
training_classes = 16
examples_per_class_per_batch = 4
train_cls = random.sample(range(37), k=training_classes)
classes_per_batch = max(16, training_classes)
target_img_size = 224

print(f"Class IDs seen during training {train_cls}")

def img_augmentation(img_batch, y, *args):
    batch_size = tf.shape(img_batch)[0]
    img_batch = tf.image.random_crop(img_batch, (batch_size,target_img_size,target_img_size,3))
    img_batch = tf.image.random_flip_left_right(img_batch)
    return img_batch, y

train_ds = tfsim.samplers.TFDatasetMultiShotMemorySampler(
    "oxford_iiit_pet",
    splits="train",
    examples_per_class_per_batch=examples_per_class_per_batch,
    classes_per_batch=classes_per_batch,
    preprocess_fn=resize,
    class_list=train_cls,
    augmenter=img_augmentation,
)

test_ds = tfsim.samplers.TFDatasetMultiShotMemorySampler(
    "oxford_iiit_pet",
    splits="test",
    total_examples_per_class=20,
    classes_per_batch=classes_per_batch,
    preprocess_fn=resize,
)

Class IDs seen during training [20, 18, 34, 26, 7, 13, 33, 27, 5, 3, 10, 29, 11, 6, 9, 24]


NameError: name 'tfsim' is not defined

In [ ]:
num_targets = 200
num_queries = 300
k = 3

queries_x, queries_y = test_ds.get_slice(0, num_queries)
targets_x, targets_y = test_ds.get_slice(num_queries, num_targets)
tsc = tfsim.callbacks.EvalCallback(
    queries_x,
    queries_y,
    targets_x,
    targets_y,
    metrics=["f1"],
    k=k,
)

val_loss = tfsim.callbacks.EvalCallback(
    queries_x,
    queries_y,
    targets_x,
    targets_y,
    metrics=["binary_accuracy"],
    known_classes=tf.constant(train_cls),
    k=k,
)

callbacks = [
    val_loss,
    tsc,
]

In [ ]:
embedding_size = 128

model = tfsim.architectures.EfficientNetSim(
    train_ds.example_shape,
    embedding_size,
    pooling="gem",
    gem_p=3.0,
)

In [ ]:
epochs = 5
LR = 0.0001
gamma = 256
steps_per_epoch = 100
val_steps = 50
distance = "cosine"

loss = tfsim.losses.CircleLoss(gamma=gamma)

brute_force_search = tfsim.search.NMSLibSearch(
    distance=distance,
    dim=embedding_size,
    method='brute_force',
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(LR),
    loss=loss,
    distance=distance,
    search=brute_force_search,
)
history = model.fit(
    train_ds,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_ds,
    validation_steps=val_steps,
    callbacks=callbacks,
)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(["loss", "val_loss"])
plt.title(f"Loss: {loss.name} - LR: {LR}")
plt.show()

In [ ]:
index_size = 360
query_size = 360
index_x, index_y = test_ds.get_slice(0, index_size)
index_data = tf.cast(index_x, dtype="int32")

test_x, test_y = test_ds.get_slice(index_size, query_size)
test_y = [int(c) for c in test_y]
test_data = tf.cast(test_x, dtype="int32")

model.reset_index()
model.index(index_x, index_y, data=index_data)

In [ ]:
num_examples = 3
num_neigboors = 5
idxs = random.sample(range(len(test_y)), num_examples)
batch = tf.gather(test_x, idxs)
nns = model.lookup(batch, k=num_neigboors)
for bid, nn in zip(idxs, nns):
    if test_y[bid] in train_cls:
        display(Markdown("**Known Class**"))
    else:
        display(Markdown("**Unknown Class**"))
    tfsim.visualization.viz_neigbors_imgs(test_data[bid], test_y[bid], nn, class_mapping=breeds, cmap="Greys")